In [ ]:
import numpy as np
import regex as re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
import tokenizers
from transformers import RobertaTokenizer, TFRobertaModel

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/FOS/Train_data.csv")

In [ ]:
data

,text,label
0,Where do ghosts play tennis? On a tennis corpse!,Pun
1,I have a few jokes about unemployed people but...,Pun
2,He fell off the slide and landed on the floor ...,Onomatopoeia
3,As suspicious as a cat,Simile
4,"Pity about poor Catherine, though. Tick-tock, ...",Onomatopoeia
...,...,...
3193,The bee buzzed in my ear.,Onomatopoeia
3194,"I thought I understood the meaning of ""When Pi...",Pun
3195,As steadfast as the sun,Simile
3196,As solitary as a tomb,Simile


In [ ]:
X_data = data[['text']].to_numpy().reshape(-1)
y_data = data[['label']].to_numpy().reshape(-1)

In [ ]:
MODEL_NAME = 'roberta-base'
MAX_LEN = 256
BATCH_SIZE = 16

In [ ]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)

        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])

        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN

        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')

        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
category_to_id = {}
category_to_name = {}

for index, c in enumerate(y_data):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name[category_id] = c

    y_data[index] = category_id

# Display dictionary
category_to_name

{0: 'Pun',
 1: 'Onomatopoeia',
 2: 'Simile',
 3: 'Metaphor',
 4: 'Alliteration',
 5: 'Personification',
 6: 'Hyperbole',
 7: 'Paradox'}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.15, random_state=777)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
n_categories=8

In [ ]:
X_train = roberta_encode(X_train, tokenizer)
X_test = roberta_encode(X_test, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [ ]:
def build_model(n_categories):
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

            # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

            # Huggingface transformers have multiple outputs, embeddings are the first one,
            # so let's slice out the first position
    x = x[1]

    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
                optimizer=tf.keras.optimizers.Adam(lr=1e-5),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'],
                device='gpu')


    return model

In [ ]:
model = build_model(n_categories)
model.summary()


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 256)]                0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                              

In [ ]:
print('Training...')
history = model.fit(X_train,
                        y_train,
                        epochs=10,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_test, y_test))

Training...
Epoch 1/10
170/170 [==============================] - 151s 888ms/step - loss: 1.6800 - accuracy: 0.2995 - val_loss: 1.7251 - val_accuracy: 0.2667
Epoch 2/10
170/170 [==============================] - 151s 889ms/step - loss: 1.6760 - accuracy: 0.3171 - val_loss: 1.6945 - val_accuracy: 0.2667
Epoch 3/10
170/170 [==============================] - 151s 889ms/step - loss: 1.6617 - accuracy: 0.3308 - val_loss: 1.7610 - val_accuracy: 0.3063
Epoch 4/10
170/170 [==============================] - 149s 875ms/step - loss: 1.6612 - accuracy: 0.3389 - val_loss: 1.7197 - val_accuracy: 0.3063
Epoch 5/10
170/170 [==============================] - 149s 876ms/step - loss: 1.6433 - accuracy: 0.3797 - val_loss: 1.7107 - val_accuracy: 0.3063
Epoch 6/10
170/170 [==============================] - 149s 877ms/step - loss: 1.6482 - accuracy: 0.3414 - val_loss: 1.6906 - val_accuracy: 0.3063
Epoch 7/10
170/170 [==============================] - 149s 877ms/step - loss: 1.6642 - accuracy: 0.3094 - val_lo